In [10]:
import cv2
import numpy as np
import os

def swap_face_with_masks(target_img_path, target_mask_path, source_img_path, source_mask_path, output_path):
    # 이미지 로드
    target_img = cv2.imread(target_img_path)
    target_mask = cv2.imread(target_mask_path)
    source_img = cv2.imread(source_img_path)
    source_mask = cv2.imread(source_mask_path)
    
    # 이미지 크기 맞추기
    target_mask = cv2.resize(target_mask, (target_img.shape[1], target_img.shape[0]))
    source_mask = cv2.resize(source_mask, (source_img.shape[1], source_img.shape[0]))
    
    # 얼굴 특징을 보존하기 위한 마스크 처리
    target_gray = cv2.cvtColor(target_mask, cv2.COLOR_BGR2GRAY)
    source_gray = cv2.cvtColor(source_mask, cv2.COLOR_BGR2GRAY)
    
    # 얼굴 특징 강조를 위한 이미지 처리
    target_edges = cv2.Canny(target_img, 100, 200)
    source_edges = cv2.Canny(source_img, 100, 200)
    
    # 마스크 영역 찾기
    _, target_mask_bin = cv2.threshold(target_gray, 30, 255, cv2.THRESH_BINARY)
    _, source_mask_bin = cv2.threshold(source_gray, 30, 255, cv2.THRESH_BINARY)
    
    # 마스크 정제
    kernel = np.ones((5,5), np.uint8)
    target_mask_bin = cv2.dilate(target_mask_bin, kernel, iterations=2)
    target_mask_bin = cv2.erode(target_mask_bin, kernel, iterations=1)
    target_mask_bin = cv2.GaussianBlur(target_mask_bin, (5,5), 0)
    
    # 얼굴 영역 좌표 찾기
    target_y, target_x = np.where(target_mask_bin > 127)
    source_y, source_x = np.where(source_mask_bin > 127)
    
    if len(target_y) == 0 or len(source_y) == 0:
        raise ValueError("마스크 영역을 찾을 수 없습니다.")
    
    # 바운딩 박스
    target_y1, target_y2 = np.min(target_y), np.max(target_y)
    target_x1, target_x2 = np.min(target_x), np.max(target_x)
    source_y1, source_y2 = np.min(source_y), np.max(source_y)
    source_x1, source_x2 = np.min(source_x), np.max(source_x)
    
    # 소스 얼굴 추출 및 리사이즈
    source_face = source_img[source_y1:source_y2, source_x1:source_x2]
    target_size = (target_x2 - target_x1, target_y2 - target_y1)
    source_face_resized = cv2.resize(source_face, target_size)
    
    # 얼굴 특징 보존을 위한 마스크 생성
    target_face_region = target_img[target_y1:target_y2, target_x1:target_x2]
    
    # Laplacian 필터를 사용하여 얼굴 특징 강조
    laplacian = cv2.Laplacian(cv2.cvtColor(target_face_region, cv2.COLOR_BGR2GRAY), cv2.CV_64F)
    feature_mask = np.abs(laplacian) > 30
    feature_mask = cv2.dilate(feature_mask.astype(np.uint8), kernel, iterations=1)
    
    # 피부톤 매칭
    source_face_lab = cv2.cvtColor(source_face_resized, cv2.COLOR_BGR2LAB)
    target_face_lab = cv2.cvtColor(target_face_region, cv2.COLOR_BGR2LAB)
    
    # L 채널은 부분적으로 유지하고 a,b 채널 조정
    source_face_lab[:,:,0] = cv2.addWeighted(source_face_lab[:,:,0], 0.7, target_face_lab[:,:,0], 0.3, 0)
    source_face_lab[:,:,1:] = target_face_lab[:,:,1:]
    source_face_matched = cv2.cvtColor(source_face_lab, cv2.COLOR_LAB2BGR)
    
    # 결과 이미지 준비
    result = target_img.copy()
    
    # 마스크 블렌딩
    mask_region = target_mask_bin[target_y1:target_y2, target_x1:target_x2] / 255.0
    mask_blur = cv2.GaussianBlur(mask_region, (15, 15), 0)
    
    # 특징 보존 블렌딩
    blended_face = np.zeros_like(target_face_region)
    for c in range(3):
        blended_face[:,:,c] = np.where(
            feature_mask,
            target_face_region[:,:,c],
            mask_blur * source_face_matched[:,:,c] + (1 - mask_blur) * target_face_region[:,:,c]
        )
    
    # 결과 적용
    result[target_y1:target_y2, target_x1:target_x2] = blended_face
    
    # 결과 저장
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, result)
    
    # 디버그 이미지 저장
    debug_dir = os.path.join(os.path.dirname(output_path), 'debug')
    os.makedirs(debug_dir, exist_ok=True)
    
    cv2.imwrite(os.path.join(debug_dir, 'target_mask.jpg'), target_mask_bin)
    cv2.imwrite(os.path.join(debug_dir, 'feature_mask.jpg'), feature_mask * 255)
    cv2.imwrite(os.path.join(debug_dir, 'source_face_matched.jpg'), source_face_matched)
    
    return result

# 실행
try:
    result = swap_face_with_masks(
        target_img_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/painter.jpeg",
        target_mask_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/Painter_output_face_detection.jpg",
        source_img_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/gonsoo_fall.jpg",
        source_mask_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/Gonsoo_output_face_detection.jpg",
        output_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/output_image/result.jpg"
    )
    print("얼굴 교체가 완료되었습니다!")
except Exception as e:
    print(f"에러 발생: {str(e)}")

얼굴 교체가 완료되었습니다!


In [19]:
import cv2
import numpy as np
import os

def check_file_exists(file_path, file_type):
    """파일 존재 여부 확인"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"{file_type} 파일을 찾을 수 없습니다: {file_path}")

def load_image(image_path, image_type):
    """이미지 로드 및 검증"""
    check_file_exists(image_path, image_type)
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"{image_type} 이미지를 로드할 수 없습니다: {image_path}")
    return img

def swap_faces_with_mask(source_img_path, target_img_path, source_mask_path, target_mask_path, output_path):
    """
    마스크 이미지를 사용하여 얼굴 교체를 수행하는 함수
    """
    try:
        # 이미지 로드 및 검증
        print("이미지 로딩 중...")
        source_img = load_image(source_img_path, "소스 이미지")
        target_img = load_image(target_img_path, "타겟 이미지")
        source_mask = load_image(source_mask_path, "소스 마스크")
        target_mask = load_image(target_mask_path, "타겟 마스크")
        
        print("이미지 크기:")
        print(f"소스 이미지: {source_img.shape}")
        print(f"타겟 이미지: {target_img.shape}")
        print(f"소스 마스크: {source_mask.shape}")
        print(f"타겟 마스크: {target_mask.shape}")
        
        # 마스크 이미지를 그레이스케일로 변환
        source_mask_gray = cv2.cvtColor(source_mask, cv2.COLOR_BGR2GRAY)
        target_mask_gray = cv2.cvtColor(target_mask, cv2.COLOR_BGR2GRAY)
        
        # 마스크 이진화
        _, source_mask_bin = cv2.threshold(source_mask_gray, 50, 255, cv2.THRESH_BINARY)
        _, target_mask_bin = cv2.threshold(target_mask_gray, 50, 255, cv2.THRESH_BINARY)
        
        # 마스크 윤곽선 찾기
        source_contours, _ = cv2.findContours(source_mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        target_contours, _ = cv2.findContours(target_mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if not source_contours or not target_contours:
            raise ValueError("마스크에서 얼굴 영역을 찾을 수 없습니다.")
        
        # 가장 큰 윤곽선 선택
        source_contour = max(source_contours, key=cv2.contourArea)
        target_contour = max(target_contours, key=cv2.contourArea)
        
        # 바운딩 박스 얻기
        source_x, source_y, source_w, source_h = cv2.boundingRect(source_contour)
        target_x, target_y, target_w, target_h = cv2.boundingRect(target_contour)
        
        print(f"소스 얼굴 영역: x={source_x}, y={source_y}, w={source_w}, h={source_h}")
        print(f"타겟 얼굴 영역: x={target_x}, y={target_y}, w={target_w}, h={target_h}")
        
        # 소스 얼굴 영역 추출
        source_face = source_img[source_y:source_y+source_h, source_x:source_x+source_w]
        source_face_mask = source_mask_bin[source_y:source_y+source_h, source_x:source_x+source_w]
        
        # 타겟 크기에 맞게 소스 얼굴 리사이즈
        source_face_resized = cv2.resize(source_face, (target_w, target_h))
        source_mask_resized = cv2.resize(source_face_mask, (target_w, target_h))
        
        # 결과 이미지 준비
        result = target_img.copy()
        
        # 타겟 마스크 영역 가져오기
        target_mask_region = target_mask_bin[target_y:target_y+target_h, target_x:target_x+target_w]
        
        # 블렌딩을 위한 마스크 준비
        mask_blur = cv2.GaussianBlur(target_mask_region.astype(np.float32), (15, 15), 0)
        mask_blur = mask_blur / 255.0
        mask_blur = np.expand_dims(mask_blur, axis=-1)
        
        # 얼굴 합성
        face_region = result[target_y:target_y+target_h, target_x:target_x+target_w]
        blended_face = (1 - mask_blur) * face_region + mask_blur * source_face_resized
        result[target_y:target_y+target_h, target_x:target_x+target_w] = blended_face
        
        # 결과 저장
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)
        cv2.imwrite(output_path, result)
        print(f"얼굴 교체가 완료되었습니다! 결과가 저장된 경로: {output_path}")
        
        # 디버그 이미지 저장
        debug_dir = os.path.join(output_dir, 'debug')
        os.makedirs(debug_dir, exist_ok=True)
        cv2.imwrite(os.path.join(debug_dir, 'source_mask.jpg'), source_mask_bin)
        cv2.imwrite(os.path.join(debug_dir, 'target_mask.jpg'), target_mask_bin)
        cv2.imwrite(os.path.join(debug_dir, 'blended_face.jpg'), blended_face)
        
        return result
        
    except Exception as e:
        print(f"에러 발생: {str(e)}")
        print(f"에러 타입: {type(e).__name__}")
        raise

# 실행
try:
    result = swap_faces_with_mask(
        source_img_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/gonsoo_fall.jpg",
        target_img_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/painter.jpeg",
        source_mask_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/Gonsoo_output_face_detection.jpg",
        target_mask_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/input_image/Painter_output_face_detection.jpg",
        output_path="/home/sagemaker-user/Self-Study-Generative-AI/05-In-Painting/output_image/result.jpg"
    )
    print("얼굴 교체가 성공적으로 완료되었습니다!")
except Exception as e:
    print(f"최종 에러: {str(e)}")

이미지 로딩 중...
이미지 크기:
소스 이미지: (4032, 3024, 3)
타겟 이미지: (1344, 768, 3)
소스 마스크: (3465, 2598, 3)
타겟 마스크: (3465, 1979, 3)
소스 얼굴 영역: x=1088, y=1601, w=697, h=1252
타겟 얼굴 영역: x=861, y=626, w=230, h=660
에러 발생: operands could not be broadcast together with shapes (660,230,1) (660,0,3) 
에러 타입: ValueError
최종 에러: operands could not be broadcast together with shapes (660,230,1) (660,0,3) 
